# Set-covering using Genetic Algorithm


In [48]:
import logging
import random
# from matplotlib import pyplot as plt
from functools import reduce
import math
from collections import deque


In [49]:
logging.getLogger().setLevel(logging.INFO)

In [50]:
N = 1000
POPULATION_SIZE = 2*N
NUM_GENERATIONS = 200
OFFSPRING_SIZE = math.ceil(1.5*N)

### Problem definition


In [51]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]


### Genetic operators and definition

In [52]:

class Individual:
    def __init__(self, problem, genome=None):
        if genome is None:
            self.genome = tuple([0]*len(problem))
        else:
            self.genome = genome
        self.problem = problem
        self.fitness = self.compute_fitness()        

    def __str__(self):
        return str([x for gene, x in zip(self.genome, self.problem) if gene == 1])

    # def is_goal(self):
    #     return self.covered() == N

    def weight(self):
        return sum((len(el) for gene, el in zip(self.genome, self.problem) if gene == 1))
    
    def covered(self):
        sol = (element for gene, element in zip(self.genome, self.problem) if gene == 1)
        return len( reduce(lambda a, b: set(a) | set(b), sol, set()) )

    def compute_fitness(self):
        # covered number / weight of the solution + bonus if it is a goal solution
        # return 0.3*self.covered() - 0.1*self.weight() #+ N*self.is_goal()
        return (self.covered(), -self.weight())

#### Genetic operators


In [53]:
def crossover(g1, g2):
    cut_point = random.randint(0, len(g1))
    return g1[:cut_point] + g2[cut_point:]

def mutation(g):
    mutation_point = random.randint(0, len(g) - 1)
    return g[:mutation_point] + (1 - g[mutation_point],) + g[mutation_point + 1:]

def tournament(population, tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)



## Genetic algorithm

### Initial Population


In [54]:
all_states = list(set([tuple(x) for x in problem(N, seed=42)]))

# logging.debug(f"All states: {all_states}")
#logging.debug(f"All states: {all_states_b}")

population = [Individual(all_states) for _ in range(POPULATION_SIZE)]

KeyboardInterrupt: 

### Evolution

In [ ]:
LOG_FREQUENCY = 10
STEADY_STATE_COUNT = 20
latest_fitness = deque(maxlen=STEADY_STATE_COUNT)

for generation in range(NUM_GENERATIONS):
    offspring = list()
    for i in range(OFFSPRING_SIZE):
        mutation_rate = 0.3
        if random.random() < mutation_rate:
            p = tournament(population)
            o = mutation(p.genome)
        else:
            p1 = tournament(population)
            p2 = tournament(population)
            o = crossover(p1.genome, p2.genome)
        offspring.append(Individual(all_states, o))
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
    best_one = population[0]
    if (generation + 1) % LOG_FREQUENCY == 0:
        logging.debug(f"#{generation + 1} fitness={best_one.fitness}, w = {best_one.weight()}") 
    latest_fitness.append(best_one.fitness)
    if latest_fitness.count(best_one.fitness) == STEADY_STATE_COUNT:
        logging.debug(f"Stopping at generation {generation + 1} with fitness: {best_one.fitness}")
        break
    


KeyboardInterrupt: 

In [ ]:
print(f"Number of Generation: {generation+1}")
print(f"#{N} : weight: {population[0].weight()}, fitness = {population[0].fitness}")

Number of Generation: 63
#1000 : weight: 3757, fitness = (1000, -3757), sol=True


In [55]:
def genetic_algorithm(N):
    POPULATION_SIZE = 2*N
    NUM_GENERATIONS = 200
    OFFSPRING_SIZE = math.ceil(1.5*N)
    
    all_states = list(set([tuple(x) for x in problem(N, seed=42)]))
    population = [Individual(all_states) for _ in range(POPULATION_SIZE)]
    latest_fitness = deque(maxlen=STEADY_STATE_COUNT)

    for generation in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            mutation_rate = 0.3
            if random.random() < mutation_rate:
                p = tournament(population)
                o = mutation(p.genome)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = crossover(p1.genome, p2.genome)
            offspring.append(Individual(all_states, o))
        population += offspring
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
        best_one = population[0]
        if (generation + 1) % LOG_FREQUENCY == 0:
            logging.debug(f"#{generation + 1} fitness={best_one.fitness}, w = {best_one.weight()}") 
        latest_fitness.append(best_one.fitness)
        if latest_fitness.count(best_one.fitness) == STEADY_STATE_COUNT:
            logging.debug(f"Stopping at generation {generation + 1} with fitness: {best_one.fitness}")
            break
    print(f"Number of Generation: {generation+1}")
    print(f"#{N} : weight: {population[0].weight()}, BLOAT= {(population[0].weight()-N)/N*100}, fitness = {population[0].fitness}")


In [56]:
for n in [5, 10, 20, 100, 500, 1000]:
    genetic_algorithm(n)
    print("----------")

Number of Generation: 23
#5 : weight: 5, BLOAT= 1.0, fitness = (5, -5)
----------
Number of Generation: 55
#10 : weight: 10, BLOAT= 1.0, fitness = (10, -10)
----------
Number of Generation: 34
#20 : weight: 29, BLOAT= 1.45, fitness = (20, -29)
----------
Number of Generation: 41
#100 : weight: 216, BLOAT= 2.16, fitness = (100, -216)
----------
Number of Generation: 61
#500 : weight: 1557, BLOAT= 3.114, fitness = (500, -1557)
----------
